## QTM 350 Project
### Data Merge and Clean with SQL

In [1]:
import pandas as pd

# Replace with your file paths
mortality = pd.read_csv("API_SH.DYN.MORT.MA_DS2_en_csv_v2_27807.csv", skiprows=4)
fertility = pd.read_csv("API_SP.ADO.TFRT_DS2_en_csv_v2_21875.csv", skiprows=4)
life_expectancy = pd.read_csv("API_SP.DYN.LE00.IN_DS2_en_csv_v2_19383.csv", skiprows=4)



In [9]:
years = [str(year) for year in range(1960, 2024)]
keep_cols = ['Country Name', 'Country Code'] + years

mortality_long = mortality[keep_cols].melt(id_vars=['Country Name', 'Country Code'], 
                                               var_name='Year', value_name='MortalityRate')

fertility_long = fertility[keep_cols].melt(id_vars=['Country Name', 'Country Code'], 
                                               var_name='Year', value_name='FertilityRate')

life_expectancy_long = life_expectancy[keep_cols].melt(id_vars=['Country Name', 'Country Code'], 
                                               var_name='Year', value_name='LifeExpectancy')


In [10]:
import sqlite3

# Create SQLite database in memory (or replace with a filename to save it to disk)
conn = sqlite3.connect(":memory:")  

# Write each long-form DataFrame as a table
mortality_long.to_sql("mortality_long", conn, index=False, if_exists='replace')
fertility_long.to_sql("fertility_long", conn, index=False, if_exists='replace')
life_expectancy_long.to_sql("life_expectancy_long", conn, index=False, if_exists='replace')


17024

In [11]:
# Merge all three long tables
query_merge = """
CREATE TABLE merged_data AS
SELECT 
    l."Country Name" AS country_name,
    l."Country Code" AS country_code,
    l."Year" AS year,
    l."LifeExpectancy",
    m."MortalityRate",
    f."FertilityRate"
FROM life_expectancy_long l
LEFT JOIN mortality_long m ON l."Country Name" = m."Country Name" AND l."Year" = m."Year"
LEFT JOIN fertility_long f ON l."Country Name" = f."Country Name" AND l."Year" = f."Year";
"""
conn.execute(query_merge)


In [12]:
query_clean_strict = """
CREATE TABLE cleaned_data AS
SELECT *
FROM merged_data
WHERE LifeExpectancy IS NOT NULL
  AND MortalityRate IS NOT NULL
  AND FertilityRate IS NOT NULL;
"""
conn.execute(query_clean_strict)


In [13]:
cleaned_df = pd.read_sql("SELECT * FROM cleaned_data", conn)
cleaned_df.head()


,country_name,country_code,year,LifeExpectancy,MortalityRate,FertilityRate
0,Afghanistan,AFG,1960,32.799000,360.5,140.598
1,United Arab Emirates,ARE,1960,50.651000,211.2,155.143
2,Antigua and Barbuda,ATG,1960,62.635000,86.8,120.289
3,Australia,AUS,1960,70.817073,27.6,43.132
4,Austria,AUT,1960,68.585610,47.7,52.005


In [14]:
income_df = pd.read_csv("/Users/noora_ni0321/Desktop/QTM 350/API_SP.ADO.TFRT_DS2_en_csv_v2_21875/Metadata_Country_API_SP.ADO.TFRT_DS2_en_csv_v2_21875.csv")

# Select only necessary columns from income_df
income_df = income_df[['Country Code', 'IncomeGroup']]

# Merge cleaned dataset with income data on 'country_code'
merged_df = pd.merge(cleaned_df, income_df,
                     left_on='country_code', right_on='Country Code', how='left')

# Drop rows where income is missing
cleaned_df = merged_df.dropna(subset=['IncomeGroup'])
cleaned_df.head()

,country_name,country_code,year,LifeExpectancy,MortalityRate,FertilityRate,Country Code,IncomeGroup
0,Afghanistan,AFG,1960,32.799000,360.5,140.598,AFG,Low income
1,United Arab Emirates,ARE,1960,50.651000,211.2,155.143,ARE,High income
2,Antigua and Barbuda,ATG,1960,62.635000,86.8,120.289,ATG,High income
3,Australia,AUS,1960,70.817073,27.6,43.132,AUS,High income
4,Austria,AUT,1960,68.585610,47.7,52.005,AUT,High income


In [15]:
cleaned_df.to_csv("cleaned_data.csv", index=False)